# Social Media Monitoring


The following table contains data on the politicians and their associated social acounts

In [ ]:
import os
from smm_wrapper import SMM
import qgrid
smm = SMM()

politicians_df = smm.dv.politicians_df()
qgrid.show_grid(politicians_df)

Please choose a party from the table above and enter its name in the field below:

In [ ]:
party = 'AfD'
aggregation = 'month' #aggregation can be by day ('day'), week ('week') or month('month')

In [ ]:
#aggregating tweets data by party affiliation
party_tweets = {'party':party,'date':[],'tweets':[], 'replies':[]}

for index, row in politicians_df.iterrows():
    if row['tw_ids'] is not None and row['affiliation'] == party:
        tw = smm.api.tweets_by(twitter_user_id=row['tw_ids'][0], aggregate_by=aggregation)
        for i, date in enumerate(tw['labels'], 0):
            if date in party_data['date']:
                curr_i = party_data['date'].index(date)
                party_data['tweets'][curr_i] += tw['values'][i]
            else:
                party_data['date'].append(date)
                party_data['tweets'].append(tw['values'][i])
                party_data['replies'].append(0)
        for i, date in enumerate(tw_rep['labels'], 0):
            if date in party_data['date']:
                curr_i = party_data['date'].index(date)
                party_data['replies'][curr_i] += tw_rep['values'][i]
            else:
                party_data['date'].append(date)
                party_data['replies'].append(tw_rep['values'][i])

In [ ]:
#aggregating replies data by party affiliation
for index, row in politicians_df.iterrows():
    if row['tw_ids'] is not None and row['affiliation'] == party:
        tw_rep = smm.api.reply_to(twitter_user_id=row['tw_ids'], aggregate_by=aggregation)
        print(tw_rep['values'])
        for i, date in enumerate(tw_rep['labels'], 0):
            if date in party_data['date']:
                curr_i = party_data['date'].index(date)
                party_data['replies'][curr_i] += tw_rep['values'][i]
            else:
                party_data['date'].append(date)
                party_data['replies'].append(tw_rep['values'][i])
                
print(party_data)

In [ ]:
#plotting data
import plotly
from plotly import graph_objs
data = sorted(zip(party_data['date'],element['tweets']))
data_rep = sorted(zip(party_data['date'],element['replies']))
x_tw, y_tw = zip(*data)
x_rep, y_rep = zip(*data_rep)
trace_tw = [graph_objs.Scatter(x=x_tw, y=y_tw, mode='lines+markers', name=party_data['party'])]
trace_rep = [graph_objs.Scatter(x=x_rep, y=y_rep, mode='lines+markers', name=party_data['party'])]

layout_tw = graph_objs.Layout(hovermode='closest', xaxis=dict(title='Date'), yaxis=dict(title='Tweets'),showlegend=True)
layout_rep = graph_objs.Layout(hovermode='closest', xaxis=dict(title='Date'), yaxis=dict(title='Replies'),showlegend=True)

plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({"data": trace_tw, "layout": layout_tw})
plotly.offline.iplot({"data": trace_rep, "layout": layout_rep})